# Library Imports

# Selecting the language pairs for training the translation model

# Data import and setup

# Building models

## Seq2Seq model

## RNN/LSTM/GRU model

# Outlining the Loss and Optimizer functions

# Training the model

# Visualising results

# Translation example

# Analysis